In [7]:
import json
import os
import re
from tqdm import tqdm

In [1]:
def clean_code_blocks() -> None:
    """
    处理数据集中所有对象，对prefix_split_*和suffix_split_*字段检查并移除
    前缀("```"或"```python")和后缀("```")，保持与annotate函数一致的遍历风格
    """
    # 文件路径配置
    input_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp.json"
    output_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_aligned.json"
    
    # 检查输入文件是否存在
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"输入文件未找到：{input_path}")
    
    # 读取数据集（列表中的每个元素是字典）
    with open(input_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    # 验证数据集格式
    if not isinstance(dataset, list):
        raise TypeError("数据集应为列表格式，每个元素应为字典")
    
    print(f"✅ 成功加载数据集，共 {len(dataset)} 条记录")
    
    # 处理所有对象
    processed_dataset = []
    for item in tqdm(dataset, desc="清理代码块标记"):
        # 复制原始对象（保留所有字段）
        cleaned_item = {}
        
        # 处理所有prefix_split字段
        prefix_fields = [key for key in item if key.startswith("prefix_split_")]
        for field in prefix_fields:
            content = item[field]
            # 检查并移除前缀
            if content.startswith("```python"):
                content = content[len("```python"):]
            elif content.startswith("```"):
                content = content[len("```"):]
            # 检查并移除后缀
            if content.endswith("```"):
                content = content[:-len("```")]
            # 去除首尾空白
            cleaned_item[field] = content
        
        # 处理所有suffix_split字段
        suffix_fields = [key for key in item if key.startswith("suffix_split_")]
        for field in suffix_fields:
            content = item[field]
            # 检查并移除前缀
            if content.startswith("```python"):
                content = content[len("```python"):]
            elif content.startswith("```"):
                content = content[len("```"):]
            # 检查并移除后缀
            if content.endswith("```"):
                content = content[:-len("```")]
            # 去除首尾空白
            cleaned_item[field] = content
        
        # 保留其他字段（如completion）
        other_fields = [key for key in item 
                       if not key.startswith("prefix_split_") 
                       and not key.startswith("suffix_split_")]
        for field in other_fields:
            cleaned_item[field] = item[field]
        
        processed_dataset.append(cleaned_item)
    
    # 保存清理后的数据集
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 清理完成！结果保存至：")
    print(f"   {output_path}")
    
    # 验证清理结果
    if processed_dataset:
        first_item = processed_dataset[0]
        # 查找示例字段
        sample_prefix = next((key for key in first_item if key.startswith("prefix_split_")), None)
        sample_suffix = next((key for key in first_item if key.startswith("suffix_split_")), None)
        
        if sample_prefix:
            print(f"\n{sample_prefix} 清理示例（前30字符）：")
            print(f"{first_item[sample_prefix][:30]}...")
        if sample_suffix:
            print(f"\n{sample_suffix} 清理示例（前30字符）：")
            print(f"{first_item[sample_suffix][:30]}...")

In [6]:
# execution
if __name__ == "__main__":
    try:
        clean_code_blocks()
    except Exception as e:
        print(f"处理过程中发生错误：{str(e)}")

✅ 成功加载数据集，共 242 条记录


清理代码块标记: 100%|██████████| 242/242 [00:00<00:00, 226264.28it/s]


✅ 清理完成！结果保存至：
   C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_cleaned.json

prefix_split_1 清理示例（前30字符）：
import asyncio
import websocke...

suffix_split_1 清理示例（前30字符）：
next_token = <LibFunc->(use ge...


In [2]:
def process_json_files(primary_path, secondary_path, sorted_output_path, unmatched_output_path):
    # 读取主要JSON文件
    try:
        with open(primary_path, 'r', encoding='utf-8') as f:
            primary_data = json.load(f)
    except Exception as e:
        print(f"读取主要文件失败: {e}")
        return
    
    # 读取次要JSON文件
    try:
        with open(secondary_path, 'r', encoding='utf-8') as f:
            secondary_data = json.load(f)
    except Exception as e:
        print(f"读取次要文件失败: {e}")
        return
    
    # 构建次要数据列表和completion到索引的映射
    completion_indices = {}
    for idx, item in enumerate(secondary_data):
        completion = item.get("completion")
        if completion and completion not in completion_indices:
            completion_indices[completion] = idx
    
    # 构建排序后的结果和记录已匹配的索引
    sorted_data = []
    matched_indices = set()
    
    # 处理主要文件中的每个项
    for index, primary_item in enumerate(primary_data):
        target_completion = primary_item.get("completion")
        if not target_completion:
            print(f"警告: 主要文件中第{index+1}个对象没有'completion'键")
            continue
            
        # 查找匹配项
        if target_completion in completion_indices:
            item_index = completion_indices[target_completion]
            sorted_data.append(secondary_data[item_index])
            matched_indices.add(item_index)
            print(f"成功: 主要文件中第{index+1}个对象匹配完成")
        else:
            # 立即打印未找到的信息
            print(f"错误: 主要文件中第{index+1}个对象的completion在次要文件中未找到")
    
    # 收集未匹配的项（保留原始顺序）
    unmatched_data = [item for idx, item in enumerate(secondary_data) if idx not in matched_indices]
    
    # 写入排序后的结果
    try:
        with open(sorted_output_path, 'w', encoding='utf-8') as f:
            json.dump(sorted_data, f, ensure_ascii=False, indent=2)
        print(f"成功生成排序后的文件: {os.path.abspath(sorted_output_path)}")
    except Exception as e:
        print(f"写入排序文件失败: {e}")
    
    # 写入未匹配的项
    try:
        with open(unmatched_output_path, 'w', encoding='utf-8') as f:
            json.dump(unmatched_data, f, ensure_ascii=False, indent=2)
        print(f"成功生成未匹配项文件: {os.path.abspath(unmatched_output_path)}")
    except Exception as e:
        print(f"写入未匹配项文件失败: {e}")

In [31]:
if __name__ == "__main__":
    # 定义文件路径
    primary_file = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split_copy.json"
    secondary_file = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp.json"
    sorted_output_file = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_sorted.json"
    unmatched_output_file = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_unmatched.json"
    
    # 执行处理操作
    process_json_files(primary_file, secondary_file, sorted_output_file, unmatched_output_file)


成功: 主要文件中第1个对象匹配完成
成功: 主要文件中第2个对象匹配完成
成功: 主要文件中第3个对象匹配完成
成功: 主要文件中第4个对象匹配完成
成功: 主要文件中第5个对象匹配完成
成功: 主要文件中第6个对象匹配完成
成功: 主要文件中第7个对象匹配完成
成功: 主要文件中第8个对象匹配完成
成功: 主要文件中第9个对象匹配完成
成功: 主要文件中第10个对象匹配完成
成功: 主要文件中第11个对象匹配完成
成功: 主要文件中第12个对象匹配完成
成功: 主要文件中第13个对象匹配完成
成功: 主要文件中第14个对象匹配完成
成功: 主要文件中第15个对象匹配完成
成功: 主要文件中第16个对象匹配完成
成功: 主要文件中第17个对象匹配完成
成功: 主要文件中第18个对象匹配完成
成功: 主要文件中第19个对象匹配完成
成功: 主要文件中第20个对象匹配完成
成功: 主要文件中第21个对象匹配完成
成功: 主要文件中第22个对象匹配完成
成功: 主要文件中第23个对象匹配完成
成功: 主要文件中第24个对象匹配完成
成功: 主要文件中第25个对象匹配完成
成功: 主要文件中第26个对象匹配完成
成功: 主要文件中第27个对象匹配完成
成功: 主要文件中第28个对象匹配完成
成功: 主要文件中第29个对象匹配完成
成功: 主要文件中第30个对象匹配完成
成功: 主要文件中第31个对象匹配完成
成功: 主要文件中第32个对象匹配完成
成功: 主要文件中第33个对象匹配完成
成功: 主要文件中第34个对象匹配完成
成功: 主要文件中第35个对象匹配完成
成功: 主要文件中第36个对象匹配完成
成功: 主要文件中第37个对象匹配完成
成功: 主要文件中第38个对象匹配完成
成功: 主要文件中第39个对象匹配完成
成功: 主要文件中第40个对象匹配完成
成功: 主要文件中第41个对象匹配完成
成功: 主要文件中第42个对象匹配完成
成功: 主要文件中第43个对象匹配完成
成功: 主要文件中第44个对象匹配完成
成功: 主要文件中第45个对象匹配完成
成功: 主要文件中第46个对象匹配完成
成功: 主要文件中第47个对象匹配完成
成功: 主要文件中第48个对象匹配完成
成功: 主要文件中第49个对象匹配完成
成功: 主要文件中第50个对象匹配完成
成功: 主要文件中

In [3]:
def compare_datasets_with_completion():
    # 文件路径配置
    file1_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp.json"
    file2_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split_copy.json"
    
    # 检查文件是否存在
    for path in [file1_path, file2_path]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"文件未找到：{path}")
    
    # 读取两个数据集
    with open(file1_path, 'r', encoding='utf-8') as f:
        dataset1 = json.load(f)  # temp_cleaned.json（数据量较少）
    
    with open(file2_path, 'r', encoding='utf-8') as f:
        dataset2 = json.load(f)  # modified_line_completion_split_copy.json（数据量较多）
    
    # 验证数据集格式
    if not isinstance(dataset1, list) or not isinstance(dataset2, list):
        raise TypeError("两个文件都应包含列表格式的JSON数据")
    
    print(f"✅ 成功加载数据集")
    print(f"  temp_cleaned.json 包含 {len(dataset1)} 条记录")
    print(f"  modified_line_completion_split_copy.json 包含 {len(dataset2)} 条记录")
    
    # 检查dataset2是否比dataset1长
    if len(dataset2) < len(dataset1):
        raise ValueError("modified_line_completion_split_copy.json 的记录数少于 temp.json")
    
    # 存储不匹配的记录
    mismatch_records = []
    
    # 遍历temp_cleaned.json中的所有记录
    for i, (item1, item2) in enumerate(tqdm(zip(dataset1, dataset2[:len(dataset1)]), 
                                          total=len(dataset1), 
                                          desc="比对字段和completion值")):
        record_mismatch = {
            "index": i,
            "field_mismatch": False,
            "completion_mismatch": False,
            "details": {}
        }
        
        # 验证当前项是否为字典
        if not isinstance(item1, dict) or not isinstance(item2, dict):
            record_mismatch["field_mismatch"] = True
            record_mismatch["details"]["reason"] = "数据项不是字典格式"
            mismatch_records.append(record_mismatch)
            continue
        
        # 1. 检查字段是否完全相同
        fields1 = set(item1.keys())
        fields2 = set(item2.keys())
        
        if fields1 != fields2:
            record_mismatch["field_mismatch"] = True
            record_mismatch["details"]["only_in1"] = sorted(fields1 - fields2)
            record_mismatch["details"]["only_in2"] = sorted(fields2 - fields1)
        
        # 2. 检查completion字段是否存在且值相同
        if "completion" in fields1 and "completion" in fields2:
            # 两个文件都有completion字段，比对值
            comp1 = item1["completion"]
            comp2 = item2["completion"]
            
            if comp1 != comp2:
                record_mismatch["completion_mismatch"] = True
                # 保存值用于显示（只保留前200字符避免输出过长）
                record_mismatch["details"]["completion1"] = comp1[:200] + ("..." if len(comp1) > 200 else "")
                record_mismatch["details"]["completion2"] = comp2[:200] + ("..." if len(comp2) > 200 else "")
        else:
            # completion字段在某个文件中缺失
            record_mismatch["completion_mismatch"] = True
            record_mismatch["details"]["reason"] = "completion字段在某个文件中缺失"
        
        # 如果有任何不匹配，添加到记录列表
        if record_mismatch["field_mismatch"] or record_mismatch["completion_mismatch"]:
            mismatch_records.append(record_mismatch)
    
    # 输出比对结果
    if not mismatch_records:
        print("\n✅ 比对完成：所有记录的字段结构和completion值完全一致！")
    else:
        print(f"\n❌ 比对完成：共发现 {len(mismatch_records)} 处不匹配")
        # 显示前5处不匹配
        for i, mismatch in enumerate(mismatch_records[:5]):
            print(f"\n--- 不匹配记录 #{mismatch['index'] + 1} ---")
            
            if mismatch["field_mismatch"]:
                print("字段不匹配：")
                print(f"  仅在temp_cleaned.json中存在：{mismatch['details']['only_in1']}")
                print(f"  仅在modified_line_completion_split_copy.json中存在：{mismatch['details']['only_in2']}")
            
            if mismatch["completion_mismatch"]:
                print("completion值不匹配：")
                if "reason" in mismatch["details"]:
                    print(f"  {mismatch['details']['reason']}")
                else:
                    print(f"  temp_cleaned.json: {mismatch['details']['completion1']}")
                    print(f"  modified_line_completion_split_copy.json: {mismatch['details']['completion2']}")
        
        # 如果有更多不匹配，提示用户
        if len(mismatch_records) > 5:
            print(f"\n... 还有 {len(mismatch_records) - 5} 处不匹配未显示")
    
    return mismatch_records

In [4]:
if __name__ == "__main__":
    try:
        mismatches = compare_datasets_with_completion()
    except Exception as e:
        print(f"比对过程中发生错误：{str(e)}")

✅ 成功加载数据集
  temp_cleaned.json 包含 242 条记录
  modified_line_completion_split_copy.json 包含 2665 条记录


比对字段和completion值: 100%|██████████| 242/242 [00:00<00:00, 747438.56it/s]


✅ 比对完成：所有记录的字段结构和completion值完全一致！
